In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *

### FNN on cleaned features

In [2]:
findex = 1
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(47383,)
907.0
907


In [3]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

47383 154


In [4]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = F.leaky_relu(self.l6(x))
        x = (self.l7(x))
        
        return x

In [5]:
max_percent = 0.76
model_no = 1
for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(30,50,5)
    
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2500):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=100,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
        min_fp = xval.size()[0]*0.2
        yval = yval.reshape(yval.shape[0],)
        train_op = mymlp(xval)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(yval,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))
        if(percent>max_percent*0.98):
            if(fp<min_fp):
                if(model_no<6):
                        max_percent = percent
                        print("min fp, max_percent",fp,percent)
                        model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(model_no)
                        torch.save(mymlp.state_dict(),model_path)
                        print("saving model on val: ",val_iter," and weight: ",w)
                        model_no = model_no + 1
                else:
                    break
        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([8693,  604,   61,  118]))
('tn, fp, fn, tp: ', array([8498,  799,   39,  140]))
('min fp, max_percent', 799, 0.7821229050279329)
('saving model on val: ', 1, ' and weight: ', 35.0)
('tn, fp, fn, tp: ', array([8512,  785,   53,  126]))
('tn, fp, fn, tp: ', array([8317,  980,   42,  137]))
('tn, fp, fn, tp: ', array([8373,  924,   48,  131]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([8836,  477,   57,  107]))
('tn, fp, fn, tp: ', array([8351,  962,   40,  124]))
('tn, fp, fn, tp: ', array([8658,  655,   48,  116]))
('tn, fp, fn, tp: ', array([8318,  995,   33,  131]))
('min fp, max_percent', 995, 0.7987804878048781)
('saving model on val: ', 2, ' and weight: ', 45.0)
('tn, fp, fn, tp: ', array([7974, 1339,   28,  136]))
('min fp, max_percent', 1339, 0.8292682926829268)
('saving model on val: ', 2, ' and weight: ', 50.0)
('val iter: ', 3)
('tn, fp, fn, tp: ', array([8424,  864,   36,  152]))
('tn, fp, fn, tp: ', array([8416,  872,   27,  1

KeyboardInterrupt: 

In [7]:
xtest_t,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest_t).cuda())
final_pred = np.zeros(ytest.shape[0],)
print final_pred.shape
for i in range(1,6):
    model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(i)
    
    
    test_model = c_mlp().cuda()
    test_model.load_state_dict(torch.load(model_path))
    test_op = test_model(xtest)
    test_op = test_op.cpu().data.numpy()
    pred_labels = np.argmax(test_op,axis=1).reshape(final_pred.shape[0],)
    print pred_labels.shape
    final_pred = pred_labels + final_pred
    print final_pred.shape
    cf = metrics.confusion_matrix(ytest,pred_labels).ravel()
    print('tn, fp, fn, tp: ',cf)

(11957,)
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10823,  1092,    21,    21]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10563,  1352,    17,    25]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10088,  1827,    12,    30]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10625,  1290,    17,    25]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10405,  1510,    16,    26]))


In [8]:
final_pred[final_pred>1] = 1
cf = metrics.confusion_matrix(ytest,final_pred).ravel()
print('tn, fp, fn, tp: ',cf)

('tn, fp, fn, tp: ', array([9552, 2363,   10,   32]))


In [ ]:
pred_labels.shape

In [ ]:
final_pred.shape